In [33]:
import os
from google.cloud import speech
from google.cloud import texttospeech as tts
import argparse
import uuid
from google.cloud import dialogflow
from IPython.display import Javascript

In [3]:
# !pip uninstall google-cloud
# !pip install --upgrade google-cloud-texttospeech
# !pip install google-cloud-dialogflow
!pip install dialogflow

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\kazij\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\google\\~rotobuf\\internal\\_api_implementation.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\kazij\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1


In [36]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'client-service-key.json'
speech_client = speech.SpeechClient()

In [68]:
def speech_to_text(mp3_filename):
    mp3_file = mp3_filename # "recording.wav"

    with open(mp3_file, 'rb') as file:
        byte_mp3 = file.read()

    audio_mp3 = speech.RecognitionAudio(content=byte_mp3)

    config_mp3 = speech.RecognitionConfig(
        sample_rate_hertz = 48000,
        enable_automatic_punctuation = True,
        language_code = 'en-US'
    )

    response_standard_mp3 = speech_client.recognize(
        config = config_mp3,
        audio = audio_mp3
    )

    return response_standard_mp3

In [40]:
print(response := speech_to_text("recording.wav"))

results {
  alternatives {
    transcript: "I forgot when my project was too and now I can\'t get away with it. Should I drop out of my classes or should I do something else? In the meantime? I\'m shopping with Tom and Jerry. They\'re both my best friend\'s and now it\'s Black Friday. Thank you."
    confidence: 0.9601263999938965
  }
}
total_billed_time {
  seconds: 30
}



In [60]:
print(response_str := (str(response.results)[31:-38]).replace("\\",""))

I forgot when my project was too and now I can't get away with it. Should I drop out of my classes or should I do something else? In the meantime? I'm shopping with Tom and Jerry. They're both my best friend's and now it's Black Friday. Thank you.


In [64]:
def text_to_wav(voice_name: str, text: str):
    language_code = "-".join(voice_name.split("-")[:2])
    text_input = tts.SynthesisInput(text=text)
    voice_params = tts.VoiceSelectionParams(
        language_code=language_code, name=voice_name
    )
    audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

    client = tts.TextToSpeechClient()
    response = client.synthesize_speech(
        input=text_input, voice=voice_params, audio_config=audio_config
    )

    filename = f"{language_code}.wav"
    with open(filename, "wb") as out:
        out.write(response.audio_content)
        print(f'Generated speech saved to "{filename}"')

In [72]:
text_to_wav("en-US-Wavenet-D", "What is the temperature in Sydney?")

Generated speech saved to "en-US.wav"


In [ ]:
# [START dialogflow_detect_intent_text]
# project id , language code, en-US
def detect_intent_texts(project_id, session_id, texts, language_code='en-US'):
    """Returns the result of detect intent with texts as inputs.
    Using the same `session_id` between requests allows continuation
    of the conversation."""
    from google.cloud import dialogflow

    session_client = dialogflow.SessionsClient()

    session = session_client.session_path(project_id, session_id)
    print("Session path: {}\n".format(session))

    for text in texts:
        text_input = dialogflow.TextInput(text=text, language_code=language_code)

        query_input = dialogflow.QueryInput(text=text_input)

        response = session_client.detect_intent(
            request={"session": session, "query_input": query_input}
        )

        print("=" * 20)
        print("Query text: {}".format(response.query_result.query_text))
        print(
            "Detected intent: {} (confidence: {})\n".format(
                response.query_result.intent.display_name,
                response.query_result.intent_detection_confidence,
            )
        )
        print("Fulfillment text: {}\n".format(response.query_result.fulfillment_text))

In [72]:
text_input = "Hello"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'client-service-key.json'
session_client = dialogflow.SessionsClient()
session = session_client.session_path("teak-vent-330619", "user_id")

print("Session path: {}\n".format(session))

text_input = dialogflow.TextInput(text=text_input, language_code="en-US")

query_input = dialogflow.QueryInput(text=text_input)

response = session_client.detect_intent(
    request={"session": session, "query_input": query_input}
)
message_response_raw = response.query_result.fulfillment_messages
print(response_clean := str(message_response_raw)[16:-4])


Session path: projects/teak-vent-330619/agent/sessions/user_id

"Hello! How can I help you?"


In [73]:
text_to_wav("en-US-Wavenet-D", response_clean)

Generated speech saved to "en-US.wav"
